# Create interactive timeseries plot for operation of selected system design for selected scenario

In [1]:
# Hack to emulate running notebook from root directory.
import os
os.chdir('..')

In [2]:
import yaml
import json
import numpy as np

import utils
from configs import get_experiment_config
from energy_model import EnergyModel
from utils import ScenarioData, solve_model, get_Gurobi_WLS_env
from utils.plotting import init_profile_fig, add_profile
import matplotlib.colors as mcolors

#### Select experiment settings, system design, and scenario to plot operation for

In [3]:
expt_id = 'two_techs'
techs = ['CAES','Li-ion']

In [4]:
techs_str = '-'.join(techs)
settings, base_params = get_experiment_config(expt_id)
available_technologies = list(settings['probability_settings']['storage'].keys())

In [5]:
design_fpath = os.path.join(*settings['results_dir'],'prior',f'{techs_str}_design.yaml')
scenario_fpaths = [os.path.join(*settings['scenarios_dir'],'thetas','scenario_0.yaml')]

In [6]:
# Load design and scenario
with open(design_fpath, 'r') as f: design_results = yaml.safe_load(f)
design = design_results['design']
scenarios = [ScenarioData.from_file(fpath) for fpath in scenario_fpaths]

In [ ]:
print(json.dumps(design,indent=4))

In [ ]:
for scenario in scenarios:
    print(scenario)

#### Set up experiment configuration

In [9]:
settings['model_settings']['storage_technologies'] = design['storage_technologies']

In [ ]:
settings['solver_settings']['verbose'] = True
settings['solver_settings']['env'] = get_Gurobi_WLS_env(silence = not settings['solver_settings']['verbose'])

#### Run operational optimisation

In [ ]:
solved_model = solve_model(scenarios, settings, design)

#### Check result correctness

In [12]:
solved_model.save_results('temp_results.yaml')
with open('temp_results.yaml', 'r') as f: op_results = yaml.safe_load(f)
os.remove('temp_results.yaml')

In [13]:
for key in ['wind_capacity','solar_capacity']:
    assert np.isclose(op_results['design'][key]['value'], design[key]['value'], rtol=1e-4),\
        f"Design values do not match for {key}."
for key in design['storage_technologies']:
    assert np.isclose(op_results['design']['storage_capacities'][key]['value'], design['storage_capacities'][key]['value'], rtol=1e-4),\
        f"Design values do not match for {key}."

In [ ]:
# Investigate operational cost relative to design optimisation
print(json.dumps(op_results['overall_objective'],indent=4))
if len(scenarios) > 1:
    print(json.dumps(op_results['scenario_objective_contributions'],indent=4))

**NOTE**: manually compare to cost from design optimisation results file.

#### Plot operation figure

TODO: generalise to plotting multiple scenarios

In [15]:
m = 0
scenario = scenarios[m]

In [16]:
d=dict(groupclick="toggleitem")

In [ ]:
colors = list(mcolors.TABLEAU_COLORS.values())

fig = init_profile_fig(y_titles={'y1':'Energy flow (MWh)', 'y2':'State of Charge (GWh)', 'y3':'Price (€/kWh)'})

fig = add_profile(fig, scenario.load[:solved_model.T]/1e3, name='Plant',
                  legendgroup='load', legendgrouptitle_text='Load',
                  visible='legendonly',
                  line=dict(color='black', width=2.5, dash='dash'), zorder=10)
fig = add_profile(fig, solved_model.grid_energies[m].solution.values/1e3, name='Grid',
                  legendgroup='load',
                  visible='legendonly',
                  line=dict(color='black', width=2.5), zorder=10)

wind = solved_model.scenarios[m].norm_wind_gen*solved_model.model.variables.wind_capacity.solution.values
fig = add_profile(fig, wind/1e3, name=f'Wind',
                  legendgroup='generation', legendgrouptitle_text='Generation',
                  line=dict(color='#0165fc', width=2.5), zorder=5)
solar = solved_model.scenarios[m].norm_solar_gen*solved_model.model.variables.solar_capacity.solution.values
fig = add_profile(fig, solar/1e3, name=f'Solar',
                  legendgroup='generation',
                  visible='legendonly',
                  line=dict(color='#fac205', width=2.5), zorder=5)
curtailment = getattr(solved_model.model.variables,f'generation_curtailment_s{m}').solution.values
fig = add_profile(fig, curtailment/1e3, name='Curtailment',
                  legendgroup='generation', visible='legendonly',
                  line=dict(color='#ff000d', width=2.5), zorder=4)

for i,tech in enumerate(solved_model.techs):
    if solved_model.model.variables[f'{tech}_capacity'].solution > 0:
        SOC = getattr(solved_model.model.variables,f'SOC_{tech}_s{m}').solution
        fig = add_profile(fig, SOC/1e6, name=tech, yaxis=f'y2',
                          legendgroup='soc', legendgrouptitle_text='Storage SOC',
                          #visible='legendonly',
                          line=dict(color=colors[available_technologies.index(tech)], width=3), zorder=8)

fig = add_profile(fig, solved_model.scenarios[m].elec_prices, name='Electricity price', yaxis='y3',
                  legendgroup='other', legendgrouptitle_text='Other',
                  #visible='legendonly',
                  line=dict(color='#ff028d', width=2.5), zorder=12)
fig = add_profile(fig, solved_model.get_flared_energy()[f's{m}']['total_energy_dump']/1e3, name='Flared energy', yaxis='y',
                  legendgroup='other', visible='legendonly',
                  line=dict(color='#be0119', width=2.5), zorder=4)

fig.update_layout(legend=dict(groupclick="toggleitem"))
fig.update_layout(hovermode="x")

fig['layout']['xaxis'].update(range=['2000-05-26','2000-06-09'])
fig.write_html(os.path.join('plots',f'{techs_str}_operation_plot.html'))
fig.write_image(os.path.join('plots',f'{techs_str}_operation_plot.pdf'), width=1800, height=600)
fig.show()

#### Compute some additional summary stats

In [ ]:
flared_energy = solved_model.get_flared_energy()
for s,d in flared_energy.items():
    print([s, [(key,float(val.sum().values)) for key,val in d.items()]])

In [ ]:
storage_cycles = solved_model.get_storage_cycles()
print(storage_cycles)